In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import panel as pn
import hvplot.xarray
import hvplot.pandas
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
import mysql.connector

pn.extension('tabulator')
pn.extension()
from sqlalchemy import create_engine

# Get dataframes from MySQL database

## helper function

In [2]:
def get_dataframe(table_name):
    con = mysql.connector.connect(host="localhost", database='moodle', user="root", passwd="",use_pure=True)

    cursor = con.cursor()
    query = "SELECT * FROM " + table_name
    cursor.execute(query)

    df = pd.DataFrame(cursor.fetchall())
    df.columns = cursor.column_names

    cursor.close()
    con.close()
    
    return df

## dataframes from database

In [3]:
user = get_dataframe('mdl_user')
quiz = get_dataframe('mdl_quiz')
quiz_attempts = get_dataframe('mdl_quiz_attempts')
quiz_slots = get_dataframe('mdl_quiz_slots')
quiz_grades = get_dataframe('mdl_quiz_grades')
quiz_feedback = get_dataframe('mdl_quiz_feedback')
question = get_dataframe('mdl_question')
question_attempts = get_dataframe('mdl_question_attempts')
question_usages = get_dataframe('mdl_question_usages')
course = get_dataframe('mdl_course')
course_modules = get_dataframe('mdl_course_modules')
course_completion = get_dataframe('mdl_course_completions')
user_enrolments = get_dataframe('mdl_user_enrolments')
enrol = get_dataframe('mdl_enrol')
course_sections = get_dataframe('mdl_course_sections')

# build own database for dashboard

In [4]:
user_data = {'Name': user['firstname'] + ' ' + user['lastname'],
                      'SuS ID': user['id']
                     }

quiz_data = {'Aufgabe': quiz['name'],
             'Quiz ID': quiz['id'],
             'Maximale Punktzahl': quiz['sumgrades'],
             'Maximale Note in %': quiz['grade']
            }

quiz_attempts_data = {'SuS ID': quiz_attempts['userid'],
                      'Quiz ID': quiz_attempts['quiz'],
                      'Frage ID': quiz_attempts['uniqueid'],
                      'Startzeit': quiz_attempts['timestart'],
                      'Abgabezeit': quiz_attempts['timefinish'],
                      'Punktzahl': quiz_attempts['sumgrades'],
                      'Anzahl Versuche': quiz_attempts['attempt']
                     }

quiz_grade_data = {'Beste Punktzahl in %': quiz_grades['grade'],
                   'SuS ID': quiz_grades['userid'],
                   'Quiz ID': quiz_grades['quiz']
                  }

user_enrolments_data = {'Anmelde ID': user_enrolments['enrolid'],
                        'SuS ID': user_enrolments['userid']
                       }

enrol_data = {'Anmelde ID': enrol['id'],
              'Klassen ID': enrol['courseid']
             }

course_data = {'Klasse': course['fullname'],
               'Klassen ID': course['id']
              }

course_sections_data = {'Klassen ID': course_sections['course'],
                        'Fach': course_sections['name']
                       }

In [5]:
pd.set_option('display.max_rows', 1000)

df_user = pd.DataFrame(user_data)

df_quiz_attempts = pd.DataFrame(quiz_attempts_data)
df_quiz_attempts['Startzeit'] = pd.to_datetime(df_quiz_attempts['Startzeit'], unit = 's')
df_quiz_attempts['Abgabezeit'] = pd.to_datetime(df_quiz_attempts['Abgabezeit'], unit = 's')
df_quiz_attempts['Bearbeitungsdauer'] = df_quiz_attempts['Abgabezeit'] - df_quiz_attempts['Startzeit']

df_quiz_grade = pd.DataFrame(quiz_grade_data)

#df_course_modules = pd.DataFrame(course_modules_data)

df_course = pd.DataFrame(course_data)

df_user_enrolments = pd.DataFrame(user_enrolments_data)

df_enrol = pd.DataFrame(enrol_data)

df_course_sections = pd.DataFrame(course_sections_data)

df_quiz = pd.DataFrame(quiz_data)

df_finished_quiz = pd.merge(df_user, df_quiz_attempts, on = 'SuS ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_quiz_grade, on = ['SuS ID', 'Quiz ID'], how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_user_enrolments, on = 'SuS ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_enrol, on = 'Anmelde ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_course, on = 'Klassen ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_course_sections, on = 'Klassen ID', how = 'left')
df_finished_quiz = pd.merge(df_finished_quiz, df_quiz, on = 'Quiz ID', how = 'left')
df_finished_quiz['Beste Punktzahl in %'] = df_finished_quiz['Beste Punktzahl in %']*10
df_finished_quiz['Maximale Note in %'] = df_finished_quiz['Maximale Note in %']*10
df_finished_quiz = df_finished_quiz.drop([0,1,2,3,4,5,6,7])

In [6]:
# convert duration to 'HH:MM:SS'
# quelle: https://stackoverflow.com/questions/71128122/pandas-how-to-convert-timedelta64ns-to-hhmmss

def sec_to_format(s):
    if(not pd.isna(s)):
        h,s = divmod(int(s),3600)
        m,s = divmod(s,60)
        return f'{h:02}:{m:02}:{s:02}'   
    return np.nan

df_finished_quiz['Bearbeitungsdauer'] = [sec_to_format(s) for s in df_finished_quiz['Bearbeitungsdauer'].dt.total_seconds()]

In [7]:
# get good data types
df_finished_quiz['Punktzahl'] = df_finished_quiz['Punktzahl'].apply(float)

# list of important unique values

In [8]:
subject_list = course_sections['name'].unique().tolist()
subject_list.remove(None)
class_list = df_finished_quiz['Klasse'].unique().tolist()
exercise_list = quiz['name'].unique().tolist()
name_list = df_finished_quiz['Name'].unique().tolist()
name_list.sort()
class_list.sort()

# add general overview over all exercises
default_exercise = ['Allgemein']
df_finished_quiz[default_exercise] = default_exercise

class_list_with_none = ['Keine Auswahl'] + class_list
name_list_with_none = ['Keine Auswahl'] + name_list

# Functionality: Create buttons and menus

In [9]:
exercise_button = pn.widgets.RadioButtonGroup(
    name='exercise_button', options = default_exercise + exercise_list, button_type='success', orientation = 'vertical')

subject_button = pn.widgets.RadioButtonGroup(
    name='Fach auswählen', options = subject_list, button_type='success')

class_selection = pn.widgets.Select(name='Klasse auswählen', options = class_list_with_none)

student_selection = pn.widgets.Select(name='Schüler*in auswählen', options = name_list_with_none)

start_date_picker = pn.widgets.DatetimePicker(name='Startdatum auswählen', value=dt.datetime(2023, 1, 1, 0, 0))

end_date_picker = pn.widgets.DatetimePicker(name='Enddatum auswählen', value=dt.datetime(2023, 12, 31, 0, 0))

# plots

## exercise count plot

In [10]:
df_finished_quiz['Anzahl'] = df_finished_quiz.groupby(['Punktzahl'])['Klasse'].transform('count')
count_int = df_finished_quiz.interactive()

count_pipeline = (
    count_int[
        (count_int['Fach'] == subject_button) &
        ((count_int['Abgabezeit'] >= start_date_picker) & (count_int['Abgabezeit'] <= end_date_picker)) &
        ((count_int['Aufgabe'] == exercise_button) | (count_int['Allgemein'] == exercise_button)) &
        ((count_int['Klasse'] == class_selection) | (count_int['Name'] == student_selection))
    ]
    .groupby(['Punktzahl'])['Anzahl'].count()
    .to_frame()
    .reset_index()
    .sort_values(by='Punktzahl')
    .reset_index(drop=True)
)

In [12]:
count_plot = count_pipeline.hvplot.scatter(x='Punktzahl', y='Anzahl', title='Anzahl der Punktzahl', xlabel = 'Punktzahl', ylabel = 'Anzahl Schüler*innen').output()

:DynamicMap   []
   :Scatter   [Punktzahl]   (Anzahl)

In [13]:
df_finished_quiz['Durchschnitt Punktzahl'] = df_finished_quiz.groupby(['Aufgabe'])['Punktzahl'].transform('mean')
df_finished_quiz['Durchschnitt Fehlversuche'] = df_finished_quiz.groupby(['Aufgabe'])['Anzahl Versuche'].transform('mean')
mean_int = df_finished_quiz.interactive()

mean_pipeline = (
    mean_int[
        (mean_int['Fach'] == subject_button) &
        ((mean_int['Startzeit'] >= start_date_picker) & (mean_int['Abgabezeit'] <= end_date_picker)) &
        ((mean_int['Aufgabe'] == exercise_button) | (mean_int['Allgemein'] == exercise_button)) &
        ((mean_int['Klasse'] == class_selection) | (mean_int['Name'] == student_selection))
    ]
    .groupby(['Aufgabe', 'Durchschnitt Fehlversuche'])['Durchschnitt Punktzahl'].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

mean_table = mean_pipeline.pipe(pn.widgets.Tabulator, pagination = 'remote', page_size = 10, theme = 'fast', hidden_columns = ['index']).output()

# Create Teacher Dashboard

In [18]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Mein Dashboard', 
    sidebar=[pn.pane.Markdown("# Auswahl"),  
             class_selection,
             student_selection,
             exercise_button, 
             pn.pane.Markdown("## Einstellungen")],
    main=[pn.Row(pn.Column(subject_button),
                 pn.Column(start_date_picker, height = 400), 
                 pn.Column(end_date_picker)),
         pn.Row(pn.Column(count_plot), 
                pn.Column(mean_table))],
          pn.Row(pn.Column(pn.pane.Matplotlib(exercise_plot[0])),
                pn.Column(pn.pane.Matplotlib(exercise_plot[1])))],
    accent = '#88d8b0'
)

template.show()

#template.servable();
#command: panel serve --port 53369 learning_analytics_dashboard.ipynb in terminal then click on link

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 2 (1923107637.py, line 15)